In [1]:
import sqlalchemy as sa
import pandas as pd
import requests,json

https://data.ontario.ca/api/3/action/datastore_search?resource_id=775ca815-5028-4e9b-9dd4-6975ff1be021&limit=5

In [2]:
resource_id ='775ca815-5028-4e9b-9dd4-6975ff1be021'
limit = 1000 


api_url = 'https://data.ontario.ca/api/3/action/datastore_search?resource_id={}&limit={}'.format(resource_id,limit)

print(api_url)

https://data.ontario.ca/api/3/action/datastore_search?resource_id=775ca815-5028-4e9b-9dd4-6975ff1be021&limit=1000


In [3]:
#Get data from website 

In [4]:
data = requests.get(api_url)

data = data.json()

In [5]:
data

{'help': 'https://data.ontario.ca/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'records_format': 'objects',
  'resource_id': '775ca815-5028-4e9b-9dd4-6975ff1be021',
  'total_estimation_threshold': None,
  'records': [{'_id': 1,
    'Date': '2020-12-16T00:00:00',
    'Agegroup': '12-17yrs',
    'At least one dose_cumulative': '0',
    'Second_dose_cumulative': '0',
    'fully_vaccinated_cumulative': '0',
    'third_dose_cumulative': '0',
    'Total population': '961857',
    'Percent_at_least_one_dose': '.0000',
    'Percent_fully_vaccinated': '.0000',
    'Percent_3doses': '.0000'},
   {'_id': 2,
    'Date': '2020-12-16T00:00:00',
    'Agegroup': '18-29yrs',
    'At least one dose_cumulative': '37',
    'Second_dose_cumulative': '0',
    'fully_vaccinated_cumulative': '0',
    'third_dose_cumulative': '0',
    'Total population': '2454802',
    'Percent_at_least_one_dose': '.0000',
    'Percent_fully_vaccinated':

In [6]:
covid_data = data['result']['records']

covid_vaccine = pd.DataFrame(covid_data)

covid_vaccine

,_id,Date,Agegroup,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses
0,1,2020-12-16T00:00:00,12-17yrs,0,0,0,0,961857,.0000,.0000,.0000
1,2,2020-12-16T00:00:00,18-29yrs,37,0,0,0,2454802,.0000,.0000,.0000
2,3,2020-12-16T00:00:00,30-39yrs,84,0,0,0,2077137,.0000,.0000,.0000
3,4,2020-12-16T00:00:00,40-49yrs,98,0,0,0,1855340,.0001,.0000,.0000
4,5,2020-12-16T00:00:00,50-59yrs,148,0,0,0,2019276,.0001,.0000,.0000
...,...,...,...,...,...,...,...,...,...,...,...
995,996,2021-03-16T00:00:00,60-69yrs,186873,44020,0,0,1813493,.1030,.0243,.0000
996,997,2021-03-16T00:00:00,70-79yrs,69263,27793,0,0,1181151,.0586,.0235,.0000
997,998,2021-03-16T00:00:00,80+,290594,84284,0,0,674976,.4305,.1249,.0000
998,999,2021-03-16T00:00:00,Adults_18plus,1033104,317374,0,0,12076175,.0855,.0263,.0000


In [7]:
covid_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   _id                           1000 non-null   int64 
 1   Date                          1000 non-null   object
 2   Agegroup                      1000 non-null   object
 3   At least one dose_cumulative  1000 non-null   object
 4   Second_dose_cumulative        1000 non-null   object
 5   fully_vaccinated_cumulative   1000 non-null   object
 6   third_dose_cumulative         1000 non-null   object
 7   Total population              1000 non-null   object
 8   Percent_at_least_one_dose     1000 non-null   object
 9   Percent_fully_vaccinated      1000 non-null   object
 10  Percent_3doses                1000 non-null   object
dtypes: int64(1), object(10)
memory usage: 86.1+ KB


In [8]:
# convert the relevant columns to numeric format
covid_vaccine['fully_vaccinated_cumulative'] = pd.to_numeric(covid_vaccine['fully_vaccinated_cumulative'], errors='coerce')
covid_vaccine['At least one dose_cumulative'] = pd.to_numeric(covid_vaccine['At least one dose_cumulative'], errors='coerce')

In [9]:
covid_vaccine = covid_vaccine.rename(columns={'Date': 'reported_date'})

In [10]:
covid_vaccine['new_doses_on_fully_vaccinated'] = covid_vaccine['fully_vaccinated_cumulative'].diff()
covid_vaccine['new_doses_at_least_one_dose'] = covid_vaccine['At least one dose_cumulative'].diff()

In [11]:
covid_vaccine.head()

,_id,reported_date,Agegroup,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses,new_doses_on_fully_vaccinated,new_doses_at_least_one_dose
0,1,2020-12-16T00:00:00,12-17yrs,0,0,0,0,961857,.0000,.0000,.0000,NaN,NaN
1,2,2020-12-16T00:00:00,18-29yrs,37,0,0,0,2454802,.0000,.0000,.0000,0.0,37.0
2,3,2020-12-16T00:00:00,30-39yrs,84,0,0,0,2077137,.0000,.0000,.0000,0.0,47.0
3,4,2020-12-16T00:00:00,40-49yrs,98,0,0,0,1855340,.0001,.0000,.0000,0.0,14.0
4,5,2020-12-16T00:00:00,50-59yrs,148,0,0,0,2019276,.0001,.0000,.0000,0.0,50.0


In [12]:
covid_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   _id                            1000 non-null   int64  
 1   reported_date                  1000 non-null   object 
 2   Agegroup                       1000 non-null   object 
 3   At least one dose_cumulative   1000 non-null   int64  
 4   Second_dose_cumulative         1000 non-null   object 
 5   fully_vaccinated_cumulative    1000 non-null   int64  
 6   third_dose_cumulative          1000 non-null   object 
 7   Total population               1000 non-null   object 
 8   Percent_at_least_one_dose      1000 non-null   object 
 9   Percent_fully_vaccinated       1000 non-null   object 
 10  Percent_3doses                 1000 non-null   object 
 11  new_doses_on_fully_vaccinated  999 non-null    float64
 12  new_doses_at_least_one_dose    999 non-null    fl

In [13]:
relevant_columns = ['reported_date', 'Agegroup','new_doses_on_fully_vaccinated','new_doses_at_least_one_dose']

covid_on_vaccine_data = covid_vaccine[relevant_columns]

covid_on_vaccine_data.head()

,reported_date,Agegroup,new_doses_on_fully_vaccinated,new_doses_at_least_one_dose
0,2020-12-16T00:00:00,12-17yrs,NaN,NaN
1,2020-12-16T00:00:00,18-29yrs,0.0,37.0
2,2020-12-16T00:00:00,30-39yrs,0.0,47.0
3,2020-12-16T00:00:00,40-49yrs,0.0,14.0
4,2020-12-16T00:00:00,50-59yrs,0.0,50.0


In [14]:
covid_on_vaccine_data.fillna(0,inplace=True)

/var/folders/fm/9pz_54hn7ynbrz17999wv77m0000gn/T/ipykernel_10604/1132111351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_on_vaccine_data.fillna(0,inplace=True)


In [15]:
covid_on_vaccine_data

,reported_date,Agegroup,new_doses_on_fully_vaccinated,new_doses_at_least_one_dose
0,2020-12-16T00:00:00,12-17yrs,0.0,0.0
1,2020-12-16T00:00:00,18-29yrs,0.0,37.0
2,2020-12-16T00:00:00,30-39yrs,0.0,47.0
3,2020-12-16T00:00:00,40-49yrs,0.0,14.0
4,2020-12-16T00:00:00,50-59yrs,0.0,50.0
...,...,...,...,...
995,2021-03-16T00:00:00,60-69yrs,0.0,41486.0
996,2021-03-16T00:00:00,70-79yrs,0.0,-117610.0
997,2021-03-16T00:00:00,80+,0.0,221331.0
998,2021-03-16T00:00:00,Adults_18plus,0.0,742510.0


In [16]:
covid_on_vaccine_data = covid_on_vaccine_data.fillna(0)
covid_on_vaccine_data[['new_doses_on_fully_vaccinated', 'new_doses_at_least_one_dose']].clip(lower=0)

,new_doses_on_fully_vaccinated,new_doses_at_least_one_dose
0,0.0,0.0
1,0.0,37.0
2,0.0,47.0
3,0.0,14.0
4,0.0,50.0
...,...,...
995,0.0,41486.0
996,0.0,0.0
997,0.0,221331.0
998,0.0,742510.0


In [17]:
covid_on_vaccine_data.head()

,reported_date,Agegroup,new_doses_on_fully_vaccinated,new_doses_at_least_one_dose
0,2020-12-16T00:00:00,12-17yrs,0.0,0.0
1,2020-12-16T00:00:00,18-29yrs,0.0,37.0
2,2020-12-16T00:00:00,30-39yrs,0.0,47.0
3,2020-12-16T00:00:00,40-49yrs,0.0,14.0
4,2020-12-16T00:00:00,50-59yrs,0.0,50.0


In [18]:
zoe_db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'zoezhu',
    'password'   : '2023!Schulich',
    'database'   : 'zoezhu_db'
}

In [19]:
zoe_db_url = sa.engine.URL.create(
    drivername = zoe_db_secret['drivername'],
    username   = zoe_db_secret['username'],
    password   = zoe_db_secret['password'],
    host       = zoe_db_secret['host'],
    port       = zoe_db_secret['port'],
    database   = zoe_db_secret['database']
)

In [20]:
pip install pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [21]:
#Create an engine

In [22]:
zoe_db_engine = sa.create_engine(zoe_db_url)

In [23]:
covid_on_vaccine_data.to_sql(
    name      = 'covid_on_vaccine_data',
    con       = zoe_db_engine,
    schema    = 'final_exam',
    if_exists = 'replace',
    index     = False,
    dtype     = {
        'reported_date':sa.types.DATE,
        'Agegroup': sa.types.INTEGER,
        'new_doses_on_fully_vaccinated': sa.types.INTEGER,
        'new_doses_at_least_one_dose': sa.types.INTEGER
    },
    method    = 'multi')

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type integer: "12-17yrs"
LINE 1: ...at_least_one_dose) VALUES ('2020-12-16T00:00:00', '12-17yrs'...
                                                             ^

[SQL: INSERT INTO final_exam.covid_on_vaccine_data (reported_date, "Agegroup", new_doses_on_fully_vaccinated, new_doses_at_least_one_dose) VALUES (%(reported_date_m0)s, %(Agegroup_m0)s, %(new_doses_on_fully_vaccinated_m0)s, %(new_doses_at_least_one_dose_m0)s), (%(reported_date_m1)s, %(Agegroup_m1)s, %(new_doses_on_fully_vaccinated_m1)s, %(new_doses_at_least_one_dose_m1)s), (%(reported_date_m2)s, %(Agegroup_m2)s, %(new_doses_on_fully_vaccinated_m2)s, %(new_doses_at_least_one_dose_m2)s), (%(reported_date_m3)s, %(Agegroup_m3)s, %(new_doses_on_fully_vaccinated_m3)s, %(new_doses_at_least_one_dose_m3)s), (%(reported_date_m4)s, %(Agegroup_m4)s, %(new_doses_on_fully_vaccinated_m4)s, %(new_doses_at_least_one_dose_m4)s), (%(reported_date_m5)s, %(Agegroup_m